In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import SimpleImputer
import clean_dataset
# Turn interactive plotting off
plt.ioff()

# read input text and put data inside a data frame
clean_dataset.clean_suppervised()
data = pd.read_csv("../data/base_prospect_suppervised.csv",encoding="ISO-8859-1")
# prospect =  pd.DataFrame(prospect)
data['risque'] = data['risque'].astype(object)
# data['ca_total_FL'] = data['ca_total_FL'].astype(object)
# data['effectif'] = data['effectif'].astype(object)

y = data['rdv']

data.head
data.dtypes

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown="ignore")

feature_names  = data.columns.values[2:-1]
X = data[feature_names]
#La liste des caisses régionales
lst_caisse=data['code_cr'].unique()

X_cat = X.select_dtypes(exclude=['float64','int64'])

# Disjonction with OneHotEncoder
# encoder.fit(X_cat)
# X_cat = encoder.transform(X_cat).toarray()

X_cat = pd.get_dummies(X_cat)
cat_columns = X_cat.columns
cat_columns

# X.dtypes

In [38]:

from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn import metrics

from sklearn.preprocessing import StandardScaler


from sklearn.impute import SimpleImputer

rdforest = RandomForestClassifier(class_weight='balanced',random_state=42)#,max_depth=5

lst_classif = [rdforest]
lst_classif_names = ['Random Forest']

def accuracy_score(lst_classif,lst_classif_names,X,y):
   

    for clf,name_clf in zip(lst_classif,lst_classif_names):
        skf = StratifiedKFold(n_splits=5,shuffle=True)
        scores = cross_val_score(clf, X, y, cv=skf)
        print("Mean cross validation score of "+name_clf+" classifier on cross-validation: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

def confusion_matrix(lst_classif,lst_classif_names,X,y):
    for clf,name_clf in zip(lst_classif,lst_classif_names):
        predicted = cross_val_predict(clf, X, y, cv=5) 
        print("Accuracy of "+name_clf+" classifier on cross-validation: %0.2f" % metrics.accuracy_score(y, predicted))
        print(metrics.confusion_matrix(y, predicted))

In [39]:
from sklearn.preprocessing import StandardScaler
# 3 - Cluster
# Normalize data
scaler = StandardScaler()
# TODO
X_num = X.select_dtypes(include=['float64','int64']).drop(columns=['chgt_dir'])
num_columns = X_num.columns

X_num_norm = scaler.fit_transform(X_num)


accuracy_score(lst_classif,lst_classif_names, X_num_norm, y)
confusion_matrix(lst_classif,lst_classif_names, X_num_norm, y)



AttributeError: 'RandomForestClassifier' object has no attribute 'cost_complexity_pruning_path'

In [ ]:
accuracy_score(lst_classif,lst_classif_names, X_cat.to_numpy(), y)
confusion_matrix(lst_classif,lst_classif_names,X_cat.to_numpy(), y)

In [ ]:
# columns_num_cat = pd.concat([X_num_norm, X_cat], axis=1).columns

X_num_cat = pd.concat([pd.DataFrame(X_num_norm), X_cat], axis=1)
accuracy_score(lst_classif,lst_classif_names, X_num_cat.to_numpy(), y)
confusion_matrix(lst_classif,lst_classif_names,X_num_cat.to_numpy(), y)


In [ ]:
from sklearn import tree

class_names = list(map(str, data['rdv'].unique()))

rdforest.fit(X_cat, y)


# Plot decision tree for one of the trees in the forest
plt.figure(figsize=(200,100))
tree.plot_tree(rdforest.estimators_[0], feature_names=cat_columns, class_names=class_names, filled=True,fontsize=40)

plt.savefig('fig/categorical_decision_tree')
plt.close()



In [ ]:


rdforest.fit(X_num, y)
# Plot decision tree for one of the trees in the forest
plt.figure(figsize=(200,100))
tree.plot_tree(rdforest.estimators_[0], feature_names=num_columns, class_names=class_names, filled=True,fontsize=40)
plt.savefig('fig/numerical_decision_tree')
plt.close()


In [ ]:

columns_num_cat = num_columns.tolist()
columns_num_cat.extend(cat_columns.tolist())

rdforest.fit(X_num_cat, y)
# Plot decision tree for one of the trees in the forest
plt.figure(figsize=(200,100))
tree.plot_tree(rdforest.estimators_[0], feature_names=columns_num_cat, class_names=class_names, filled=True,fontsize=40)
plt.savefig('fig/categorical_numerical_decision_tree')
plt.close()
